# CVE Database Statistics & Analysis

This notebook provides interactive visualizations and insights from the CVE database using the `cvecli` library.

**Data sources:**
- CVE records from the GitHub CVE database
- CVSS metrics (v2.0, v3.0, v3.1, v4.0)
- CWE mappings
- Affected products and vendors

In [54]:
# Setup: Import libraries and load data using cvecli
import polars as pl
import plotly.express as px

from cvecli.core.config import get_config
from cvecli.services.search import CVESearchService
from cvecli.models.parquet_models import load_all_dataframes

# Initialize config and service
config = get_config()
search_service = CVESearchService(config)

# Load all data using typed loaders
data = load_all_dataframes(config.data_dir)

# Filter out 2026 data (reserved CVEs for future years)
# Extract year from cve_id (format: CVE-YYYY-NNNN)
MAX_YEAR = 2025  # Ignore CVEs from 2026 and beyond
cve_ids_filtered = data.cves.filter(
    pl.col("cve_id").str.extract(r"CVE-(\d{4})-", 1).cast(pl.Int32) <= MAX_YEAR
)
valid_cve_ids = cve_ids_filtered.select("cve_id")

# Apply filter to all DataFrames
cves_df = cve_ids_filtered
descriptions_df = data.descriptions.join(valid_cve_ids, on="cve_id", how="semi")
metrics_df = data.metrics.join(valid_cve_ids, on="cve_id", how="semi")
products_df = data.products.join(valid_cve_ids, on="cve_id", how="semi")
versions_df = data.versions.join(valid_cve_ids, on="cve_id", how="semi")
cwes_df = data.cwes.join(valid_cve_ids, on="cve_id", how="semi")
references_df = data.references.join(valid_cve_ids, on="cve_id", how="semi")
credits_df = data.credits.join(valid_cve_ids, on="cve_id", how="semi")

print(f"Database loaded from: {config.data_dir}")
print(f"\n📊 Database Overview (filtered to ≤{MAX_YEAR}):")
print(f"  • CVE records: {len(cves_df):,}")
print(f"  • Descriptions: {len(descriptions_df):,}")
print(f"  • Metrics entries: {len(metrics_df):,}")
print(f"  • Products: {len(products_df):,}")
print(f"  • Version ranges: {len(versions_df):,}")
print(f"  • CWE mappings: {len(cwes_df):,}")
print(f"  • References: {len(references_df):,}")
print(f"  • Credits: {len(credits_df):,}")

Database loaded from: /Users/romain/dev/cyber/cvecli/data

📊 Database Overview (filtered to ≤2025):
  • CVE records: 328,179
  • Descriptions: 321,419
  • Metrics entries: 305,494
  • Products: 537,922
  • Version ranges: 1,028,499
  • CWE mappings: 338,381
  • References: 1,990,476
  • Credits: 65,173


## 1. CVE Trends Over Time

Analyzing CVE publication patterns by year and month to understand vulnerability disclosure trends.

In [55]:
# Extract year from CVE ID and analyze trends
cves_with_year = cves_df.with_columns([
    pl.col("cve_id").str.extract(r"CVE-(\d{4})-", 1).alias("year"),
    pl.col("date_published").str.slice(0, 7).alias("month_published")
]).filter(pl.col("year").is_not_null())

# CVEs by year
yearly_counts = (
    cves_with_year
    .group_by("year")
    .agg(pl.len().alias("count"))
    .sort("year")
    .filter(pl.col("year").cast(pl.Int32) >= 1999)
)

fig = px.bar(
    yearly_counts,
    x="year",
    y="count",
    title="📈 CVE Count by Year",
    labels={"year": "Year", "count": "Number of CVEs"},
    color="count",
    color_continuous_scale="Reds"
)
fig.update_layout(showlegend=False, xaxis_tickangle=-45)
fig.show()

In [56]:
# Monthly trend for recent years (heatmap)
recent_cves = cves_with_year.filter(
    (pl.col("year").cast(pl.Int32) >= 2020) & 
    (pl.col("month_published").is_not_null()) &
    (pl.col("month_published") != "")
).with_columns([
    pl.col("month_published").str.slice(5, 2).alias("month")
])

monthly_heatmap = (
    recent_cves
    .group_by(["year", "month"])
    .agg(pl.len().alias("count"))
    .sort(["year", "month"])
)

# Pivot for heatmap
heatmap_pivot = monthly_heatmap.pivot(
    on="month", 
    index="year", 
    values="count"
).fill_null(0)

# Get month columns in order
month_cols = [f"{i:02d}" for i in range(1, 13)]
available_months = [m for m in month_cols if m in heatmap_pivot.columns]

# Convert to numpy array for imshow
heatmap_values = heatmap_pivot.select(available_months).to_numpy()

fig = px.imshow(
    heatmap_values,
    x=["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"][:len(available_months)],
    y=heatmap_pivot["year"].to_list(),
    title="🗓️ CVE Publication Heatmap (Monthly, 2020+)",
    labels={"x": "Month", "y": "Year", "color": "CVE Count"},
    color_continuous_scale="YlOrRd",
    aspect="auto"
)
fig.show()

## 2. Severity Distribution

Analysis of CVE severity levels based on CVSS scores. The service uses the best available metric (preferring CNA source and newer CVSS versions).

In [57]:
# Get best CVSS score per CVE (using cvecli's preference logic)
def get_severity_label(score: float) -> str:
    if score >= 9.0:
        return "Critical"
    elif score >= 7.0:
        return "High"
    elif score >= 4.0:
        return "Medium"
    elif score >= 0.1:
        return "Low"
    else:
        return "None"

# Get best metric per CVE (prefer CNA, prefer newer CVSS version)
best_metrics = (
    metrics_df
    .filter(pl.col("base_score").is_not_null() & pl.col("metric_type").str.starts_with("cvss"))
    .with_columns([
        pl.when(pl.col("source") == "cna").then(100).otherwise(0).alias("source_pref"),
        pl.when(pl.col("metric_type") == "cvssV4_0").then(40)
         .when(pl.col("metric_type") == "cvssV3_1").then(30)
         .when(pl.col("metric_type") == "cvssV3_0").then(20)
         .otherwise(10).alias("version_pref"),
    ])
    .with_columns((pl.col("source_pref") + pl.col("version_pref")).alias("preference"))
    .sort(["cve_id", "preference"], descending=[False, True])
    .group_by("cve_id")
    .first()
    .with_columns(pl.col("base_score").map_elements(get_severity_label, return_dtype=pl.Utf8).alias("severity"))
)

severity_counts = (
    best_metrics
    .group_by("severity")
    .agg(pl.len().alias("count"))
)

# Define colors for severity levels
severity_order = ["Critical", "High", "Medium", "Low", "None"]
severity_colors = {"Critical": "#d62728", "High": "#ff7f0e", "Medium": "#ffbb78", "Low": "#98df8a", "None": "#2ca02c"}

# Sort by severity order using polars
severity_order_map = {s: i for i, s in enumerate(severity_order)}
severity_df = severity_counts.with_columns(
    pl.col("severity").replace(severity_order_map).alias("sort_order")
).sort("sort_order").drop("sort_order")

fig = px.pie(
    severity_df,
    values="count",
    names="severity",
    title="🎯 CVE Severity Distribution (Based on Best CVSS Score)",
    color="severity",
    color_discrete_map=severity_colors,
    hole=0.4
)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [58]:
# CVSS Score distribution (histogram)
fig = px.histogram(
    best_metrics.filter(pl.col("base_score") > 0),
    x="base_score",
    nbins=40,
    title="📊 CVSS Base Score Distribution",
    labels={"base_score": "CVSS Base Score", "count": "Number of CVEs"},
    color_discrete_sequence=["#e74c3c"]
)
fig.add_vline(x=4.0, line_dash="dash", line_color="orange", annotation_text="Medium")
fig.add_vline(x=7.0, line_dash="dash", line_color="darkorange", annotation_text="High")
fig.add_vline(x=9.0, line_dash="dash", line_color="red", annotation_text="Critical")
fig.update_layout(bargap=0.1)
fig.show()

In [59]:
# Severity trend over time
severity_by_year = (
    best_metrics
    .with_columns(pl.col("cve_id").str.extract(r"CVE-(\d{4})-", 1).alias("year"))
    .filter(pl.col("year").is_not_null())
    .group_by(["year", "severity"])
    .agg(pl.len().alias("count"))
    .sort("year")
    .filter(pl.col("year").cast(pl.Int32) >= 2015)
)

fig = px.bar(
    severity_by_year,
    x="year",
    y="count",
    color="severity",
    title="📈 CVE Severity Trend Over Time",
    labels={"year": "Year", "count": "Number of CVEs", "severity": "Severity"},
    color_discrete_map=severity_colors,
    category_orders={"severity": severity_order}
)
fig.update_layout(barmode="stack", xaxis_tickangle=-45)
fig.show()

## 3. Top Vendors and Products

Identifying the most frequently affected vendors and products in the CVE database.

In [60]:
# Top 20 vendors by CVE count
top_vendors = (
    products_df
    .filter(pl.col("vendor").is_not_null() & (pl.col("vendor") != "") & (pl.col("vendor") != "n/a"))
    .group_by("vendor")
    .agg(pl.col("cve_id").n_unique().alias("cve_count"))
    .sort("cve_count", descending=True)
    .head(20)
)

fig = px.bar(
    top_vendors,
    x="cve_count",
    y="vendor",
    orientation="h",
    title="🏢 Top 20 Vendors by CVE Count",
    labels={"vendor": "Vendor", "cve_count": "Number of CVEs"},
    color="cve_count",
    color_continuous_scale="Reds"
)
fig.update_layout(yaxis={'categoryorder':'total ascending'}, showlegend=False)
fig.show()

In [61]:
# Top 20 products by CVE count
top_products = (
    products_df
    .filter(pl.col("product").is_not_null() & (pl.col("product") != "") & (pl.col("product") != "n/a"))
    .group_by(["vendor", "product"])
    .agg(pl.col("cve_id").n_unique().alias("cve_count"))
    .sort("cve_count", descending=True)
    .head(20)
    .with_columns((pl.col("vendor") + " / " + pl.col("product")).alias("full_name"))
)

fig = px.bar(
    top_products,
    x="cve_count",
    y="full_name",
    orientation="h",
    title="📦 Top 20 Products by CVE Count",
    labels={"full_name": "Vendor / Product", "cve_count": "Number of CVEs"},
    color="cve_count",
    color_continuous_scale="Blues"
)
fig.update_layout(yaxis={'categoryorder':'total ascending'}, showlegend=False)
fig.show()

In [62]:
# Top vendors with severity breakdown (treemap)
vendor_severity = (
    products_df
    .filter(pl.col("vendor").is_not_null() & (pl.col("vendor") != "") & (pl.col("vendor") != "n/a"))
    .select(["cve_id", "vendor"])
    .unique()
    .join(best_metrics.select(["cve_id", "severity"]), on="cve_id", how="inner")
    .group_by(["vendor", "severity"])
    .agg(pl.len().alias("count"))
)

# Filter to top 15 vendors
top_vendor_names = top_vendors.head(15)["vendor"].to_list()
vendor_severity_top = vendor_severity.filter(pl.col("vendor").is_in(top_vendor_names))

fig = px.treemap(
    vendor_severity_top,
    path=["vendor", "severity"],
    values="count",
    title="🌳 Top 15 Vendors - Severity Breakdown",
    color="severity",
    color_discrete_map=severity_colors
)
fig.show()

## 4. CWE (Weakness) Analysis

Common Weakness Enumeration (CWE) analysis reveals the most frequent types of vulnerabilities.

In [63]:
# Top 20 CWEs
top_cwes = (
    cwes_df
    .filter(pl.col("cwe_id").is_not_null() & (pl.col("cwe_id") != "NVD-CWE-noinfo") & (pl.col("cwe_id") != "NVD-CWE-Other"))
    .group_by("cwe_id")
    .agg(pl.col("cve_id").n_unique().alias("cve_count"))
    .sort("cve_count", descending=True)
    .head(20)
)

# Common CWE descriptions
cwe_names = {
    "CWE-79": "Cross-site Scripting (XSS)",
    "CWE-89": "SQL Injection",
    "CWE-787": "Out-of-bounds Write",
    "CWE-125": "Out-of-bounds Read",
    "CWE-20": "Improper Input Validation",
    "CWE-416": "Use After Free",
    "CWE-22": "Path Traversal",
    "CWE-352": "Cross-Site Request Forgery",
    "CWE-476": "NULL Pointer Dereference",
    "CWE-78": "OS Command Injection",
    "CWE-190": "Integer Overflow",
    "CWE-119": "Buffer Overflow",
    "CWE-862": "Missing Authorization",
    "CWE-200": "Information Exposure",
    "CWE-287": "Improper Authentication",
    "CWE-434": "Unrestricted File Upload",
    "CWE-94": "Code Injection",
    "CWE-400": "Resource Exhaustion",
    "CWE-863": "Incorrect Authorization",
    "CWE-120": "Classic Buffer Overflow",
}

top_cwes_named = top_cwes.with_columns(
    pl.col("cwe_id").replace(cwe_names, default=pl.col("cwe_id")).alias("cwe_name")
)

fig = px.bar(
    top_cwes_named,
    x="cve_count",
    y="cwe_name",
    orientation="h",
    title="🔓 Top 20 CWEs (Vulnerability Types)",
    labels={"cwe_name": "CWE", "cve_count": "Number of CVEs"},
    color="cve_count",
    color_continuous_scale="Purples",
    text="cwe_id"
)
fig.update_layout(yaxis={'categoryorder':'total ascending'}, showlegend=False)
fig.update_traces(textposition='inside')
fig.show()

/var/folders/9n/4blwg2rd7k7d3z9x4y_824rw0000gn/T/ipykernel_11013/4148142547.py:36: DeprecationWarning:

the `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
(Deprecated in version 1.0.0)



In [64]:
# CWE trend over time (top 5 CWEs)
top_5_cwes = top_cwes.head(5)["cwe_id"].to_list()

cwe_yearly = (
    cwes_df
    .filter(pl.col("cwe_id").is_in(top_5_cwes))
    .with_columns(pl.col("cve_id").str.extract(r"CVE-(\d{4})-", 1).alias("year"))
    .filter(pl.col("year").is_not_null() & (pl.col("year").cast(pl.Int32) >= 2015))
    .group_by(["year", "cwe_id"])
    .agg(pl.col("cve_id").n_unique().alias("cve_count"))
    .sort("year")
)

fig = px.line(
    cwe_yearly,
    x="year",
    y="cve_count",
    color="cwe_id",
    title="📈 Top 5 CWEs Trend Over Time",
    labels={"year": "Year", "cve_count": "Number of CVEs", "cwe_id": "CWE"},
    markers=True
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()

## 5. CVSS Metrics Deep Dive

Analyzing attack vectors, complexity, and impact metrics from CVSS data.

In [65]:
# Attack vector distribution (from CVSS v3.x metrics)
cvss3_metrics = metrics_df.filter(
    pl.col("metric_type").str.starts_with("cvssV3") & 
    pl.col("attack_vector").is_not_null()
)

attack_vector_counts = (
    cvss3_metrics
    .group_by("attack_vector")
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
)

av_colors = {"NETWORK": "#e74c3c", "LOCAL": "#3498db", "ADJACENT_NETWORK": "#f39c12", "PHYSICAL": "#2ecc71"}

fig = px.pie(
    attack_vector_counts,
    values="count",
    names="attack_vector",
    title="🌐 Attack Vector Distribution (CVSS v3.x)",
    color="attack_vector",
    color_discrete_map=av_colors,
    hole=0.3
)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [66]:
# CVSS v3.x Impact breakdown (sunburst chart)
impact_data = (
    cvss3_metrics
    .filter(
        pl.col("confidentiality_impact").is_not_null() &
        pl.col("integrity_impact").is_not_null() &
        pl.col("availability_impact").is_not_null()
    )
    .group_by(["confidentiality_impact", "integrity_impact", "availability_impact"])
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
    .head(30)  # Top 30 combinations
)

fig = px.sunburst(
    impact_data,
    path=["confidentiality_impact", "integrity_impact", "availability_impact"],
    values="count",
    title="☀️ CVSS Impact Distribution (C/I/A)",
    color="count",
    color_continuous_scale="RdYlGn_r"
)
fig.show()

In [67]:
# Attack complexity vs Privileges required (heatmap)
ac_pr_data = (
    cvss3_metrics
    .filter(
        pl.col("attack_complexity").is_not_null() &
        pl.col("privileges_required").is_not_null()
    )
    .group_by(["attack_complexity", "privileges_required"])
    .agg(pl.len().alias("count"))
)

ac_pr_pivot = ac_pr_data.pivot(
    on="privileges_required",
    index="attack_complexity",
    values="count"
).fill_null(0)

pr_order = ["NONE", "LOW", "HIGH"]
available_pr = [p for p in pr_order if p in ac_pr_pivot.columns]

# Convert to numpy array for imshow
heatmap_data = ac_pr_pivot.select(available_pr).to_numpy()

fig = px.imshow(
    heatmap_data,
    x=available_pr,
    y=ac_pr_pivot["attack_complexity"].to_list(),
    title="🔐 Attack Complexity vs Privileges Required",
    labels={"x": "Privileges Required", "y": "Attack Complexity", "color": "CVE Count"},
    color_continuous_scale="Oranges",
    text_auto=True
)
fig.show()

## 6. CVE Assigners (CNAs)

Analysis of CVE Numbering Authorities (CNAs) who assign CVE identifiers.

In [68]:
# Top CVE assigners
top_assigners = (
    cves_df
    .filter(pl.col("assigner_short_name").is_not_null() & (pl.col("assigner_short_name") != ""))
    .group_by("assigner_short_name")
    .agg(pl.len().alias("cve_count"))
    .sort("cve_count", descending=True)
    .head(15)
)

fig = px.bar(
    top_assigners,
    x="assigner_short_name",
    y="cve_count",
    title="🏛️ Top 15 CVE Numbering Authorities (CNAs)",
    labels={"assigner_short_name": "CNA", "cve_count": "CVEs Assigned"},
    color="cve_count",
    color_continuous_scale="Viridis"
)
fig.update_layout(xaxis_tickangle=-45, showlegend=False)
fig.show()

## 7. Reference Sources

Analysis of where CVE reference links point to, identifying common vulnerability disclosure platforms.

In [69]:
# Extract domain from reference URLs
import re

def extract_domain(url: str) -> str:
    if not url:
        return "unknown"
    match = re.search(r'https?://(?:www\.)?([^/]+)', url)
    return match.group(1) if match else "unknown"

reference_domains = (
    references_df
    .filter(pl.col("url").is_not_null())
    .with_columns(pl.col("url").map_elements(extract_domain, return_dtype=pl.Utf8).alias("domain"))
    .group_by("domain")
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
    .head(20)
)

fig = px.bar(
    reference_domains,
    x="count",
    y="domain",
    orientation="h",
    title="🔗 Top 20 Reference Domains",
    labels={"domain": "Domain", "count": "Number of References"},
    color="count",
    color_continuous_scale="Teal"
)
fig.update_layout(yaxis={'categoryorder':'total ascending'}, showlegend=False)
fig.show()

## 8. CVSS Version Adoption

Tracking the adoption of different CVSS versions over time.

In [70]:
# CVSS version distribution
cvss_version_counts = (
    metrics_df
    .filter(pl.col("metric_type").str.starts_with("cvss"))
    .group_by("metric_type")
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
)

version_labels = {
    "cvssV2_0": "CVSS v2.0",
    "cvssV3_0": "CVSS v3.0",
    "cvssV3_1": "CVSS v3.1",
    "cvssV4_0": "CVSS v4.0"
}

cvss_version_counts = cvss_version_counts.with_columns(
    pl.col("metric_type").replace(version_labels, default=pl.col("metric_type")).alias("version_label")
)

fig = px.pie(
    cvss_version_counts,
    values="count",
    names="version_label",
    title="📊 CVSS Version Distribution",
    color_discrete_sequence=px.colors.qualitative.Set2,
    hole=0.4
)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

/var/folders/9n/4blwg2rd7k7d3z9x4y_824rw0000gn/T/ipykernel_11013/255981794.py:18: DeprecationWarning:

the `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
(Deprecated in version 1.0.0)



In [71]:
# CVSS version adoption over time
cvss_by_year = (
    metrics_df
    .filter(pl.col("metric_type").str.starts_with("cvss"))
    .with_columns(pl.col("cve_id").str.extract(r"CVE-(\d{4})-", 1).alias("year"))
    .filter(pl.col("year").is_not_null() & (pl.col("year").cast(pl.Int32) >= 2010))
    .with_columns(
        pl.col("metric_type").replace(version_labels, default=pl.col("metric_type")).alias("version_label")
    )
    .group_by(["year", "version_label"])
    .agg(pl.len().alias("count"))
    .sort("year")
)

fig = px.area(
    cvss_by_year,
    x="year",
    y="count",
    color="version_label",
    title="📈 CVSS Version Adoption Over Time",
    labels={"year": "Year", "count": "Number of Metrics", "version_label": "CVSS Version"},
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()

/var/folders/9n/4blwg2rd7k7d3z9x4y_824rw0000gn/T/ipykernel_11013/3562784507.py:8: DeprecationWarning:

the `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
(Deprecated in version 1.0.0)



## 9. Critical CVEs Analysis

A deep dive into the most critical vulnerabilities (CVSS >= 9.0).

In [72]:
# Critical CVEs (score >= 9.0) breakdown
critical_cves = best_metrics.filter(pl.col("base_score") >= 9.0)
critical_cve_ids = critical_cves["cve_id"].to_list()

print(f"🚨 Critical CVEs (CVSS >= 9.0): {len(critical_cves):,}")
print(f"   Percentage of all scored CVEs: {len(critical_cves) / len(best_metrics) * 100:.1f}%")

# Top CWEs in critical CVEs
critical_cwes = (
    cwes_df
    .filter(pl.col("cve_id").is_in(critical_cve_ids))
    .filter(pl.col("cwe_id").is_not_null() & (pl.col("cwe_id") != "NVD-CWE-noinfo"))
    .group_by("cwe_id")
    .agg(pl.col("cve_id").n_unique().alias("count"))
    .sort("count", descending=True)
    .head(10)
    .with_columns(
        pl.col("cwe_id").replace(cwe_names, default=pl.col("cwe_id")).alias("cwe_name")
    )
)

fig = px.bar(
    critical_cwes,
    x="count",
    y="cwe_name",
    orientation="h",
    title="⚠️ Top 10 CWEs in Critical CVEs",
    labels={"cwe_name": "CWE", "count": "Number of Critical CVEs"},
    color="count",
    color_continuous_scale="Reds",
    text="cwe_id"
)
fig.update_layout(yaxis={'categoryorder':'total ascending'}, showlegend=False)
fig.update_traces(textposition='inside')
fig.show()

🚨 Critical CVEs (CVSS >= 9.0): 13,840
   Percentage of all scored CVEs: 9.9%


/var/folders/9n/4blwg2rd7k7d3z9x4y_824rw0000gn/T/ipykernel_11013/2421956984.py:18: DeprecationWarning:

the `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
(Deprecated in version 1.0.0)



In [73]:
# Top vendors with critical CVEs
critical_vendors = (
    products_df
    .filter(pl.col("cve_id").is_in(critical_cve_ids))
    .filter(pl.col("vendor").is_not_null() & (pl.col("vendor") != "") & (pl.col("vendor") != "n/a"))
    .group_by("vendor")
    .agg(pl.col("cve_id").n_unique().alias("critical_count"))
    .sort("critical_count", descending=True)
    .head(15)
)

fig = px.bar(
    critical_vendors,
    x="vendor",
    y="critical_count",
    title="🔴 Top 15 Vendors with Critical CVEs",
    labels={"vendor": "Vendor", "critical_count": "Critical CVE Count"},
    color="critical_count",
    color_continuous_scale="Reds"
)
fig.update_layout(xaxis_tickangle=-45, showlegend=False)
fig.show()

## 10. Summary Statistics

Overall database statistics and key metrics.

In [74]:
# Create summary dashboard
from IPython.display import Markdown, display

# Calculate key statistics
total_cves = len(cves_df)
cves_with_metrics = len(best_metrics)
unique_vendors = products_df.filter(pl.col("vendor").is_not_null() & (pl.col("vendor") != ""))["vendor"].n_unique()
unique_products = products_df.filter(pl.col("product").is_not_null() & (pl.col("product") != ""))["product"].n_unique()
unique_cwes = cwes_df.filter(pl.col("cwe_id").is_not_null())["cwe_id"].n_unique()
avg_score = best_metrics["base_score"].mean()
median_score = best_metrics["base_score"].median()

# Severity breakdown using pure polars
severity_summary = best_metrics.group_by("severity").agg(pl.len().alias("count"))
severity_dict = {row["severity"]: row["count"] for row in severity_summary.iter_rows(named=True)}

# Date range
years = cves_with_year.filter(pl.col("year").is_not_null())["year"].unique().sort()
year_range = f"{years[0]} - {years[-1]}"

summary_md = f"""
### 📊 Database Summary

| Metric | Value |
|--------|-------|
| **Total CVE Records** | {total_cves:,} |
| **CVEs with CVSS Scores** | {cves_with_metrics:,} ({cves_with_metrics/total_cves*100:.1f}%) |
| **Year Range** | {year_range} |
| **Unique Vendors** | {unique_vendors:,} |
| **Unique Products** | {unique_products:,} |
| **Unique CWE Types** | {unique_cwes:,} |
| **Average CVSS Score** | {avg_score:.2f} |
| **Median CVSS Score** | {median_score:.2f} |

### 🎯 Severity Breakdown

| Severity | Count | Percentage |
|----------|-------|------------|
| Critical | {severity_dict.get('Critical', 0):,} | {severity_dict.get('Critical', 0)/cves_with_metrics*100:.1f}% |
| High | {severity_dict.get('High', 0):,} | {severity_dict.get('High', 0)/cves_with_metrics*100:.1f}% |
| Medium | {severity_dict.get('Medium', 0):,} | {severity_dict.get('Medium', 0)/cves_with_metrics*100:.1f}% |
| Low | {severity_dict.get('Low', 0):,} | {severity_dict.get('Low', 0)/cves_with_metrics*100:.1f}% |
"""

display(Markdown(summary_md))


### 📊 Database Summary

| Metric | Value |
|--------|-------|
| **Total CVE Records** | 328,179 |
| **CVEs with CVSS Scores** | 139,613 (42.5%) |
| **Year Range** | 1999 - 2025 |
| **Unique Vendors** | 24,207 |
| **Unique Products** | 71,130 |
| **Unique CWE Types** | 753 |
| **Average CVSS Score** | 6.72 |
| **Median CVSS Score** | 6.70 |

### 🎯 Severity Breakdown

| Severity | Count | Percentage |
|----------|-------|------------|
| Critical | 13,840 | 9.9% |
| High | 49,700 | 35.6% |
| Medium | 69,666 | 49.9% |
| Low | 6,398 | 4.6% |
